In [ ]:
import sys

# setting path
sys.path.append('..')

In [ ]:
# local package
import classified

In [ ]:
# outside package
import requests
import pandas as pd
import time
import json

In [ ]:
def use_api(address, params=None, method="get"):
    if params is None:
        params = {"api_key": classified.API_KEY}
    else:
        params["api_key"] = classified.API_KEY
    method = getattr(requests, method)
    return method(address, params=params).json()

In [ ]:
demo1 = use_api("https://api.opendota.com/api/proMatches")
demo1 = pd.DataFrame(demo1)

In [ ]:
demo1.head()

In [ ]:
demo2 = use_api("https://api.opendota.com/api/matches/{}".format(6897941675))

In [ ]:
demo2

In [ ]:
demo3 = use_api("https://api.opendota.com/api/heroes/1/matches")

In [ ]:
demo3

Get Pro Matches


In [ ]:
pro_matches = []

In [ ]:
for i in range(0, 10000, 1000):
    result = use_api(
        "https://api.opendota.com/api/explorer",
        params={"sql":
                    """
                        SELECT
                            matches.match_id
                        FROM
                            matches
                            JOIN match_patch USING ( match_id )
                            JOIN leagues USING ( leagueid )
                            JOIN player_matches USING ( match_id )
                            JOIN heroes ON heroes.id = player_matches.hero_id
                            LEFT JOIN notable_players ON notable_players.account_id = player_matches.account_id
                            LEFT JOIN teams USING ( team_id )
                        WHERE
                        TRUE
                            AND match_patch.patch >= '7.28'
                            AND leagues.tier = 'professional'
                        GROUP BY
                            matches.match_id
                        ORDER BY
                            matches.match_id NULLS LAST
                            LIMIT 1000 OFFSET {}
                    """.format(i)}
    )
    pro_matches += result["rows"]
    print("First {} data fetched.".format(i + 1000))
    time.sleep(0.5)

In [ ]:
pro_matches = [i["match_id"] for i in pro_matches]

In [ ]:
data = []

In [ ]:
for i in pro_matches:
    kept_keys = ["match_id", "radiant_win", "region", "patch", "picks_bans"]
    result = use_api("https://api.opendota.com/api/matches/{match_id}".format(match_id=i))
    data.append({k: result[k] for k in kept_keys})
    time.sleep(0.5)

In [ ]:
with open("../data/raw_data/data.json", "w") as f:
    json.dump({"data": data}, f)
